In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from SeaquestEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Seaquest-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/seaquest/S1/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(18)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.05666284 0.05265621 0.0546331  0.05646965 0.05442696 0.0566367
  0.05294273 0.05835253 0.05399445 0.05433405 0.05962743 0.05699699
  0.05386858 0.05376935 0.05578875 0.05666091 0.05612954 0.05604931]]
[[0.05613806 0.06216836 0.05888303 0.04965985 0.05236086 0.05428071
  0.05732181 0.04977066 0.05360468 0.05787463 0.05588237 0.05582144
  0.05898311 0.05625803 0.05524934 0.05940643 0.05463598 0.05170062]]
[[0.05613806 0.06216836 0.05888303 0.04965985 0.05236086 0.05428071
  0.05732181 0.04977066 0.05360468 0.05787463 0.05588237 0.05582144
  0.05898311 0.05625803 0.05524934 0.05940643 0.05463598 0.05170062]]
[[0.05613806 0.06216836 0.05888303 0.04965985 0.05236086 0.05428071
  0.05732181 0.04977066 0.05360468 0.05787463 0.05588237 0.05582144
  0.05898311 0.05625803 0.05524934 0.05940643 0.05463598 0.05170062]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = True, save_path=TB_DIR, replay_ratio=0):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", 50, 0.995, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), 50, 0.995, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [5]:
tf.reset_default_graph()
start_method(replay_ratio=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/seaquest/S1/checkpoints/model825673.cptk
worker_1worker_2  using  2  per online step
worker_3  using  2  per online step
worker_4  using  2  per online step
worker_5  using  2  per online step
worker_6  using  2  per online step
worker_7  using  2  per online step
  using  2  per online step
worker_8  using  2  per online step
worker_9  using  2  per online step
worker_10  using  2  per online stepworker_11  using  2  per online step

worker_12worker_13  using  2  per online step
worker_14  using  2  per online step
  using  2worker_15  using  2   per online step per online step



/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  826661  Running/Max:  9.600000000000001 87.0  Frames: 62
pi: [[0.04964753 0.05310572 0.03397971 0.04210471 0.05740649 0.11142366
  0.09101352 0.03278084 0.04486941 0.03191039 0.0306841  0.05703097
  0.06507321 0.0438893  0.08240764 0.04349839 0.06794829 0.06122619]]
Saving Model
Worker  worker_eval At  827663  Running/Max:  17.64 90.0  Frames: 124
pi: [[0.03524139 0.03484949 0.01234996 0.01464365 0.07031293 0.3575378
  0.04241991 0.00865126 0.03488236 0.02195467 0.00821694 0.02739011
  0.10146385 0.06179586 0.01294395 0.01006522 0.03702027 0.10826028]]
Saving Model
Worker  worker_eval At  828670  Running/Max:  24.676000000000002 90.0  Frames: 186
pi: [[0.0388053  0.04743683 0.02442493 0.01712979 0.10164215 0.24087277
  0.06293243 0.01833391 0.02346043 0.02386689 0.01546971 0.0322456
  0.13484663 0.03993526 0.0268854  0.02077707 0.02401331 0.10692158]]
Saving Model
Worker  worker_eval At  829672  Running/Max:  30.9084 90.0  Frames: 248
pi: [[0.04366167 0.03980353

Worker  worker_eval At  885726  Running/Max:  83.41972201866605 90.0  Frames: 1794
pi: [[0.07164334 0.03693561 0.00206875 0.01293069 0.04591439 0.34546474
  0.01033493 0.00134638 0.0624975  0.03228589 0.00121412 0.02729678
  0.05099492 0.03113691 0.00563057 0.00128299 0.03742052 0.223601  ]]
Saving Model
Worker  worker_eval At  888756  Running/Max:  84.07774981679945 90.0  Frames: 1857
pi: [[0.08801805 0.0409041  0.01961698 0.0149521  0.17235509 0.10277344
  0.03881906 0.01248687 0.01711471 0.02774879 0.01188901 0.03019464
  0.12297656 0.02199154 0.01979003 0.01860634 0.01239915 0.22736357]]
Saving Model
Worker  worker_eval At  900423  Running/Max:  84.57151165480212 90.0  Frames: 2100
pi: [[0.09918959 0.09315329 0.02815787 0.02200595 0.16684829 0.14550142
  0.07866768 0.01181311 0.02193682 0.01312799 0.01628341 0.05865888
  0.06085229 0.03609185 0.02231193 0.01350191 0.01719242 0.09470534]]
Saving Model
Worker  worker_eval At  930278  Running/Max:  86.52738822969539 90.0  Frames: 2723

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1322, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1307, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1409, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/juull100/Bachelor/Bachelor/ACER_ACTUALLY/Worker.py", line 28, in work_acer
    b_states, b_actions, b_rewa

KeyboardInterrupt: 

In [ ]:
TB_DIR = "saves/seaquest/final_R4/"
tf.reset_default_graph()
start_method(replay_ratio=4)

In [7]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [8]:
tf.reset_default_graph()
test_method()

INFO:tensorflow:Restoring parameters from saves/breakout/final_R1/checkpoints/model1981886.cptk


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Exception in thread Thread-38:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", li